# **Enunciado**

### Autor José Daniel Sarmiento Blanco 2192232

Design a heat exchanger to be used as a crude oil cooler. 120 kg/s of crude oil enters to the heat exchanger at 102°C and leaves at 65°C. The coolant to be used is city water entering the tube side at 21°C with a flow rate of 65 kg/s. Pressure drop for the crude oil is limited to 150 kPa.

In [107]:
caudal_de_crudo             = 120 # kg/sxx
temperatura_entrada_crudo   = 102 # ºC
temperatura_salida_crudo    = 65  # ºC

temperatura_entrada_agua    = 21  # ºC
caudal_agua                 = 65  # kg/s

presion_limite_crudo        = 150_000 # Pa

In [108]:
from pyfluids import Fluid, FluidsList, Input, Mixture
from core import ThermoFormules
import numpy as np

![](./diagrama.jpeg)

## Suposicion

[presión agua](https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra.)


- Considere como material el acero al carbono, k = 60 W/m*k.
- Diámetro de tubo de 3/4 in ( 19 mm exterior y 16 mm interior ).
- Longitud de 4m

In [109]:
k_material                            = 60    # W/ m*k
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m
longitud                              = 4 #m               

In [110]:
presion_agua = (2) * 101_325 # Pa
presion_crudo = 4136_000 # Pa

In [111]:
temperatura_propiedades_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2
print(temperatura_propiedades_crudo) #+ 273.15)

cp_propiedades_crudo =  0.83 * 1000  # J/kg k

83.5


$$ \dot{Q} = (\dot{m} *C_{p})_{crudo} * (T_{c1} -T_{c2})$$

In [112]:
calor = caudal_de_crudo * cp_propiedades_crudo * (temperatura_entrada_crudo - temperatura_salida_crudo ) 
calor

3685200.0

In [113]:
water = Fluid(FluidsList.Water)
water.name

Water

### Suponga una temperatura de propiedades de agua de

In [114]:
state_agua = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(27),
)
state_agua.as_dict()

{'compressibility': 0.0014679646038685545,
 'conductivity': 0.6097949184333273,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.5612223522704,
 'dynamic_viscosity': 0.0008508970492459952,
 'enthalpy': 113375.23765002219,
 'entropy': 395.1237635503346,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 113171.88837776403,
 'kinematic_viscosity': 8.538331917406425e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.8331192360367865,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1502.0731022326067,
 'specific_heat': 4180.301802554013,
 'specific_volume': 0.0010034506436439627,
 'surface_tension': None,
 'temperature': 27.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [115]:
cp_agua = state_agua.specific_heat
cp_agua

4180.301802554013

$$ \dot{Q} = (\dot{m} *C_{p})_{agua} * (T_{w1} - T_{w2})$$

In [116]:
temperatura_salida_agua = temperatura_entrada_agua + calor / (caudal_agua * cp_agua)
temperatura_salida_agua

34.56250990795588

In [117]:
temperatura_agua_propiedades = (temperatura_entrada_agua + temperatura_salida_agua) / 2
temperatura_agua_propiedades

27.78125495397794

In [118]:
state_agua = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(temperatura_agua_propiedades),
)
state_agua.as_dict()

{'compressibility': 0.0014644738083190445,
 'conductivity': 0.6110269720320316,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.3433167890219,
 'dynamic_viscosity': 0.000836367771464936,
 'enthalpy': 116641.02661706795,
 'entropy': 405.9901508078167,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 116437.6328705613,
 'kinematic_viscosity': 8.394373278483474e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.721638142997106,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1504.02450259156,
 'specific_heat': 4180.0692815493885,
 'specific_volume': 0.001003670103617258,
 'surface_tension': None,
 'temperature': 27.781254953977964,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [119]:
densidad_agua               = state_agua.density
k_agua                      = state_agua.conductivity
cp_agua                     = state_agua.specific_heat
prandt_agua                 = state_agua.prandtl
viscosidad_dinamica_agua     = state_agua.dynamic_viscosity

In [120]:
u_m_agua = caudal_agua / (np.pi * (diametro_interior / 2)**2 * densidad_agua)
u_m_agua

324.4699620179904

In [121]:
reynolds_agua = densidad_agua * u_m_agua * diametro_interior / viscosidad_dinamica_agua
reynolds_agua

6184522.917982205

In [122]:
f = (1.58 * np.log(reynolds_agua) - 3.28)**(-2)
f

0.0021780269861284398

In [123]:
nusselt_agua = (f / 2) * reynolds_agua * prandt_agua / (1 + 8.7*(f/2)**0.5 * (prandt_agua - 1))
nusselt_agua

16359.123058430469

In [124]:
h_i = nusselt_agua * k_agua / diametro_interior
h_i

624741.5892182598

### Resultados de la evaluación de diseño

- Material = Acero al carbono k = 60 W/mk
- Longitud del intercambiador = 4 m
- Numero de tubos = 640
- Coeficientes de convección
    - Agua : 5071.76 W/mk
    - Crudo : 2458.236773857342


    

- Diámetro de la carcasa: 0.7 m
- La distribución de los tubos en cuadrada
- Un Pitch_Size de 0.0254 m

- Temperaturas de trabajo
    - Agua :
        - Entrada : 21 º C
        - Salida  : 34.5 º C
    - Crudo :
        - Entrada : 102 ºC
        - Salida  : 65  ºC


## Referencias
- Abdulagatov, I.M., Abdulagatova, Z.Z., Grigor’ev, B.A. et al. Thermal Diffusivity, Heat Capacity, and Thermal Conductivity of Oil Reservoir Rock at High Temperatures. Int J Thermophys 42, 135 (2021). https://doi.org/10.1007/s10765-021-02878-x
- Seguros, V. (2021) Presión del Agua en Casa: Problemas habituales: Blog Verti, Verti Blog. Available at: https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra. (Accessed: 13 October 2023). 
- Bell, I.H. et al. (2014) ‘Pure and pseudo-pure fluid thermophysical property evaluation and the open-source Thermophysical Property Library coolprop’, Industrial &amp;amp; Engineering Chemistry Research, 53(6), pp. 2498–2508. doi:10.1021/ie4033999. 